In [1]:
%matplotlib inline
import matplotlib

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import discopt
import pandas as pd
import numpy as np
import copy
import sklearn
import sklearn.datasets
import xgboost as xgb
import scipy
from scipy import stats

In [4]:
n_jobs = 3
nrep = 20

# Prepare data 

In [5]:
np.random.seed(0)
digits = sklearn.datasets.load_digits()
x = digits.data
y = digits.target

ntrain = 100
order = np.random.permutation(y.size)
tr = order[:ntrain]
val = order[ntrain:]
ytr = y[tr]
xtr = x[tr, :]
yval = y[val]
xval = x[val, :]


# HYPEROPT (TPE) 

In [6]:
from hyperopt import hp, Trials, fmin, tpe, rand

In [7]:
def obj_func(kwargs, verbose=1, n_jobs=n_jobs):
    # uses global xtr, ytr, xval, yval
    kwargs = xgb_parse_params(kwargs)
    obj = -clf_acc(xgb.sklearn.XGBClassifier(nthread=n_jobs,**kwargs), xtr, ytr, xval, yval)
    if verbose > 0:
        print 'obj', obj, 'arguments', kwargs, '\n'
    return obj

In [8]:
def xgb_parse_params(kwargs):
    # parse transformed xgb arguments to usual xgb arguments
    kwargs = copy.deepcopy(kwargs)
    if 'max_depth' in kwargs:
        kwargs['max_depth'] = int(kwargs['max_depth'])

    if ('lr_trees_ratio' in kwargs) and ('n_estimators' in kwargs):
        kwargs['learning_rate'] = kwargs['lr_trees_ratio'] / kwargs['n_estimators']
        del kwargs['lr_trees_ratio']

    if 'n_estimators' in kwargs:
        kwargs['n_estimators'] = int(kwargs['n_estimators'])

    for name in kwargs:
        if name.startswith('log_'):
            new_name = name[4:]
            kwargs[new_name] = np.exp(kwargs[name])
            del kwargs[name]
    return kwargs


In [9]:
def clf_acc(clf, xtr, ytr, xte, yte):
    clf.fit(xtr, ytr)
    pred = clf.predict(xte)
    return (yte == pred).mean()

In [10]:
space_hyperopt = {
    'max_depth': hp.quniform('max_depth', 1, 20, 1),
    'lr_trees_ratio': hp.uniform('lr_trees_ratio', 2, 20),
    'n_estimators': hp.quniform('n_estimators', 50, 300, 1),
    'log_gamma': hp.uniform('log_gamma', np.log(0.01), np.log(10)),
    'log_reg_lambda': hp.uniform('log_reg_lambda', np.log(0.01), np.log(10)),
    'subsample': hp.uniform('subsample', 0.2, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.2, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.2, 1),
}

In [11]:
# algo = tpe.rand
algo = tpe.suggest
trials = Trials() # this object will keep info about runed experiments
best = fmin(fn= obj_func,
            space=space_hyperopt,
            algo=tpe.suggest,
            max_evals=nrep,
            trials=trials)        

obj -0.842663523866 arguments {'colsample_bytree': 0.35084370607626475, 'colsample_bylevel': 0.3677917417779124, 'learning_rate': 0.03645497099578443, 'n_estimators': 261, 'subsample': 0.3341861608115664, 'reg_lambda': 1.0343302779360379, 'max_depth': 18, 'gamma': 0.26988326928061263} 

obj -0.761343547437 arguments {'colsample_bytree': 0.9074991058431388, 'colsample_bylevel': 0.6097948907592092, 'learning_rate': 0.09786512824782642, 'n_estimators': 136, 'subsample': 0.3946760185856406, 'reg_lambda': 0.018909474246751136, 'max_depth': 4, 'gamma': 2.9204663060036711} 

obj -0.840895698291 arguments {'colsample_bytree': 0.9107968080623745, 'colsample_bylevel': 0.3977832572858533, 'learning_rate': 0.03755159839456701, 'n_estimators': 245, 'subsample': 0.3159469668689108, 'reg_lambda': 0.61100298507230277, 'max_depth': 11, 'gamma': 0.11747475106797176} 

obj -0.835592221567 arguments {'colsample_bytree': 0.32092914588167654, 'colsample_bylevel': 0.40555414795458855, 'learning_rate': 0.0469

In [12]:
best

{'colsample_bylevel': 0.38889922769173807,
 'colsample_bytree': 0.7664269543270454,
 'log_gamma': -0.1608709882791688,
 'log_reg_lambda': -1.813240450459376,
 'lr_trees_ratio': 2.14078618003617,
 'max_depth': 6.0,
 'n_estimators': 110.0,
 'subsample': 0.7615346067775921}

In [19]:
# more results are in trails object

In [13]:
list(trials)[0]

{'book_time': datetime.datetime(2017, 4, 24, 14, 54, 5, 508000),
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'colsample_bylevel': [0],
   'colsample_bytree': [0],
   'log_gamma': [0],
   'log_reg_lambda': [0],
   'lr_trees_ratio': [0],
   'max_depth': [0],
   'n_estimators': [0],
   'subsample': [0]},
  'tid': 0,
  'vals': {'colsample_bylevel': [0.3677917417779124],
   'colsample_bytree': [0.35084370607626475],
   'log_gamma': [-1.309765749465646],
   'log_reg_lambda': [0.033754142816365906],
   'lr_trees_ratio': [9.514747429899737],
   'max_depth': [18.0],
   'n_estimators': [261.0],
   'subsample': [0.3341861608115664]},
  'workdir': None},
 'owner': None,
 'refresh_time': datetime.datetime(2017, 4, 24, 14, 54, 5, 813000),
 'result': {'loss': -0.8426635238656452, 'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 0,
 'version': 0}